In [2]:


# from run_lx import run_lxterminal
# run_lxterminal("ros2 launch turtlebot3_gazebo turtlebot3_world.launch.py")
# run_lxterminal("ros2 run turtlebot3_teleop teleop_keyboard")



In [1]:


import rclpy
from rclpy.node import Node


from geometry_msgs.msg import Twist

class TurtleMover(Node): #here we define new node
    def __init__(self): 
        super().__init__('turtle_mover')
        #we add publisher
        self.publisher_ = self.create_publisher(Twist, 
                                                'turtle1/cmd_vel',
                                                10)

    
    def move_turtle(self,twist_command): #we add helper function
        self.publisher_.publish(twist_command)



In [2]:


rclpy.init()



In [3]:


turtle_mover = TurtleMover()



In [4]:
# run_lxterminal("ros2 launch turtlebot3_navigation2 navigation2.launch.py use_sim_time:=True map:=/home/ubuntu/turtlebot3_ws/src/jupyter_notebooks/map.yaml")

In [5]:
from geometry_msgs.msg import PoseWithCovarianceStamped

send_start_pose_publisher = turtle_mover.create_publisher(PoseWithCovarianceStamped, 'initialpose', 1)


In [6]:
initial_pose = PoseWithCovarianceStamped()
initial_pose.header.stamp = turtle_mover.get_clock().now().to_msg()
initial_pose.header.frame_id = "map"
initial_pose.pose.pose.position.x = 0.08
initial_pose.pose.pose.position.y = 0.0
initial_pose.pose.pose.orientation.z = 0.0
initial_pose.pose.pose.orientation.w = 1.0

initial_pose.pose.covariance[0] = 0.25
initial_pose.pose.covariance[7] = 0.25
initial_pose.pose.covariance[-1] = 0.06

send_start_pose_publisher.publish(initial_pose)

In [10]:
#from geometry_msgs.action import 
from nav2_msgs.action import NavigateToPose
from rclpy.action import ActionClient
from geometry_msgs.msg import PoseStamped

navigate_action = ActionClient(
    turtle_mover, 
    NavigateToPose,
    '/navigate_to_pose')


def print_feedback_goal(future):
    try:
        print("feedback {}".format(future.feedback))
    except:
        pass
    
def history_success(future):
    print("done")
    print("we have achieved the goal {}, {}".format(
        future.result(),
        future.result().result))
    
def received_task(future):
    goal_handle =  future.result()
    
    if not goal_handle.accepted:
        print("not accepted")
        return
    else:
        print("accepted goal")
    
    get_result_future = goal_handle.get_result_async()
    
    get_result_future.add_done_callback(history_success)


navigate_pose_goal = NavigateToPose.Goal()    
goal_pose = PoseStamped()
goal_pose.header.stamp = turtle_mover.get_clock().now().to_msg()
goal_pose.pose.position.x = 3.9
goal_pose.pose.position.y = 0.0
goal_pose.pose.orientation.z = 1.7
goal_pose.pose.orientation.w = 1.71
navigate_pose_goal.pose = goal_pose
goal_future_pose = navigate_action.send_goal_async(navigate_pose_goal,feedback_callback=print_feedback_goal)
goal_future_pose.add_done_callback(received_task) 

In [ ]:
rclpy.spin(turtle_mover)    

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=99, nanosec=994000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=3.7026413150532482, y=-0.03515376595442854, z=0.009996958375739464), orientation=geometry_msgs.msg.Quaternion(x=-0.0003498265930484396, y=-0.00029387757098866385, z=0.7045076556723137, w=0.7096962409062265))), navigation_time=builtin_interfaces.msg.Duration(sec=6, nanosec=355000000), number_of_recoveries=0, distance_remaining=0.20071417093276978)
accepted goal
accepted goal
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=99, nanosec=994000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=3.7026413150532482, y=-0.03515376595442854, z=0.009996958375739464), orientation=geometry_msgs.msg.

In [8]:
big_turtle_publisher = turtle_mover.create_publisher(
    Twist,
    'cmd_vel',
    10)

new_msg =  Twist()

#new_msg.linear.x = 0.3
new_msg.angular.z = 0.1
big_turtle_publisher.publish(new_msg)